# ASTR3110 Computer Laboratory 3: Classifying Images using Artificial and Convolutional Neural Networks.

In this Lab, you will be using imaging from the [CORNISH](https://cornish.leeds.ac.uk/public/index.php) survey to classify images using Neural Networks. The CORNISH survey aimed to understand massive star formation in our Galaxy by searching for ultra-compact HII regions (regions that have stars with mass > 8 times the mass of our Sun that are ionising the surrounding gas, which emit at radio wavelengths) in a portion of the plane of the disk of the Galaxy using the [Very Large Array Radio telescope](http://www.vla.nrao.edu). 

While this survey successfully discovered many new HII regions, other sources such as planetary nebulae (PNE) and background galaxies (RGs) also emit at radio wavelengths, and so were also detected in the survey data. By design, the CORNISH survey targetted a region that also contained observations in the mid-infrared taken during another survey with the Spitzer Space telescope ([the GLIMPSE survey](https://irsa.ipac.caltech.edu/data/SPITZER/GLIMPSE/)). In particular, the 4.5, 5.8, and 8.0 $\mu$m Spitzer images allow us to distinguish HII regions, PNE, and RGs due to their differnt appearance (see top panel in the below image: leftmost is a PNE, middle a HII region, and right a RG).

![SegmentLocal](CORNISH_image.png)

In the first part of the lab, you will train and test an Artificial Neural Network classifier using 300 8.0 $\mu$m images (100 each for HII, PNE, and RG sources).

In the second part of the lab, you will train and test a Convolutional Neural Network classifier, using the same sample as in Part One, but adding the 4.5 and 5.8 $\mu$m images. 

In both parts, you will need to run tests to determine the performance of your classifier, and tweak hyperparameters  in order to improve the performance. (N.B.: The term "hyperparameter" is reserved for those parameters that are set manually, e.g., the number of components in GMM, or the learning rate of your neural network. Normal parameters are determined from the data, e.g., the intercept and slope of a straight line fit)

## **If you are using google Colab, you may wish to switch to using a GPU hardware accelerator, as this can improve the speed of the Neural Networks. To do so, go to the "Edit" dropdown, click "Notebook Settings", and select "GPU" for the hardware accelerator. This needs to be done before you start coding!**

At the completion of this lab, you will have acquired (or improved) the following skills:
- Reading in fits images using astropy.
- Manipulating 2D image arrays to prepare them for input into the ANN and CNN architectures (using the ndimage and numpy packages).
- Using the keras packages for setting up and running ANN and CNN. classifiers.
- Using Scikit Learn and other packages to assess the performance of ANN and CNN classifiers.

You will be required to submit **two separate files** via iLearn (see the lab submission portal in the Assessments tab) by **11:55pm Friday 6th June 2025**. The files are:

1. **Your Colab notebook containing your code.** We will randomly select code cells to inspect and will mark them based on the following criteria, each worth **10 marks**:
    - **Comments: Your code must be commented and your comments should demonstrate that you understand what your code is doing, and why! You can insert comments by either adding explanation text placed in blocks just before code blocks OR as comments within the code blocks themselves. The marking rubric is as follows:**

            8.5-10: Thorough comments that clearly explain what and why, demonstrating excellent understanding of code.
            7.5-8.5: Comments that explain what and mostly why, demonstrate good understanding.
            6.5-7.5: Comments that mostly explain what, not why, mostly demonstrate understanding.
            5.0-6.5: Some comments explaining what or why, some demonstration of understanding.
            < 5.0: Minimal comments.

    - **Formatting of your code. Rubric:**

            8.5-10: Code is clear and easy to understand with sensible variable names.
            7.5-8.5: Mostly clear, understandable with semi-sensible variable names.
            6.5-7.5: Code is understandable OR good variable names.
            5-6.5: Some effort in formatting code OR using sensible variable names.
            < 5.: Spaghetti code, remnants of code bits that failed to run.

2. **A document (preferrably in pdf format) that contains the plots, explanation and justification  of experiments to design a better classifier, and comments on choice on a final best classifier and performance results <font color='blue'>highlighted in blue throughout the notebbook.</font>** We will mark those based on the following criteria, again worth **10 marks** each:
    - **Plots: Your plots should be well presented and explained. Rubric:**

            8.5-10: All requested plots are included and are accompanied by a caption that thoroughly explains them and details what they show and why plotted. Plots have clear symbols, clearly labeled axes, and axis ranges are set so that trends visible.
            7.5-8.5: All requested plots included and accompanied by captions that mostly explain what and why. Plots include good labels, ranges, and symbols.
            6.5-7.5: Most requested plots are included and captions mostly explain what. Good labels, ranges, and symbols. 
            5-6.5: Most requested plots included, captions contain some explanation. Adequate labels, ranges, and symbols.
            < 5.0: Many missing plots, minimal effort in explaining or making sure plots/labels are clear.

    - **Explanations and justificatons of experiments to design a better classifier. Rubric:**
    
            8.5-10: Complete, thorough explanation and justification of how experiment helps with designing classifier.
            7.5-8.5: Complete explanation and justification that mostly explains and justifies experiments.
            6.5-7.5: Explained OR justified experiments.
            5-6.5: Some effort at one of explaining or jusitifying experiments.
            <5: No/minimal effort at explaining or justifying experiments.

    - **Comments on choice on a final best classifier and performance results. Rubric:**
    
            8.5-10: Excellent explanation of choice of classifier that is driven by the results of the experiments.
            7.5-8.5: Good explanation of choice, and of results.
            6.5-7.5: Adequate explanation of choice and results.
            5.-6.5: Some effort at explaining choice OR results.
            <5: No/minimal effort 


# Part 1: Artificial Neural Networks
In this part, we will use the 8.0 $\mu$m images to design a classifier using the keras backend to build ANNs in a similar manner to that described in the [lectorial on ANNs](https://github.com/MQ-ASTR3110/ASTR3110_Tutorial_Notebooks_2024/blob/main/Solution_Notebooks/ASTR3110_Tutorial_9_ANNs.ipynb). First, the data must be read in and manipulated into a format that is accepted by the keras models.

## 1.1 Accessing the data.

You should clone the Github repository to your Google Drive as per the usual method [described here](https://github.com/MQ-ASTR3110/ASTR3110_Tutorial_Notebooks_2024/blob/main/SETUP_COLAB.md). **Please clone into a new directory so that you do not overwrite existing Labs!!**

As outlined above, the data that will be used in this lab consist of Spitzer imaging in the 4.5, 5.8, and 8.0 $\mu$m bands. The images have been uploaded to the [Github repository](https://github.com/MQ-ASTR3110/ASTR3110_Practical_Notebooks_2024/), and are stored the main ```Cornish_data``` directory, which contains three subdirectories: one each for ```HII```, ```PNE```, and ```RG``` sources. Within each of these subdirectories, there are 300 "fits" files: 3 files for each source, where the filename gives the Galactic coordinates, and the 3 files are for the different Spitzer bands (I2 = 4.5 $\mu$m, I3 = 5.8 $\mu$m, and I4 = 8.0 $\mu$m). 

##  1.2 Getting to know the data

The fits format is commonly used for storing astronomical data, and can store binary tables, images, cubes, and other formats (the format is also used by Garmin, Strava and other activity trackers), as well as coordinate and information for the image in a header. Fits files can be read into numpy arrays using the [astropy.io.fits](https://docs.astropy.org/en/stable/io/fits/#) package. 

Using the [astropy.io.fits.getdata()](https://docs.astropy.org/en/stable/io/fits/api/files.html#astropy.io.fits.getdata) function, read the image data for one source from each of the HII, PNE, and RG folders. At this point, we only require access to the 8.0 $\mu$m band images (labelled \*_I4.fits). Using numpy functions, determine basic statistics for the images (min, max pixel values, and shape of the array). 

<font color='blue'> **(1a)** Plot the images for the three sources to convince yourself that they appear to be different sources.</font> 

Because the images are relatively large, we want to resize them in order to decrease the runtime when we run our neural networks. Use the [scipy.ndimage.zoom](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.zoom.html) function to bin the images to a coarser pixel scale. Plot the resulting coarsely binned image and compare with the original for each of the three sources. Try a few different 'zoom' factors and choose a suitable factor by which to reduce the size of your images. Note, you do not want to bin too much, otherwise you lose too much information. 

<font color='blue'> **(1b)** Plot images for 2 zoom values (including your final chosen value) for the same three sources plotted in (1a) to convince yourself that they appear different.</font> 


## 1.3 Preparing the data for the ANN

As you know from lectorial 9, as input to the ANN we require the images to be flattened to 1D arrays, and also normalised so that the pixel values are between 0 and 1. We also require labels for the sources (both single digit labels as well as the one-hot vectors of binarized labels), and a training and testing/validation sample. 

To achieve some of the above, write a function that:
- accepts a directory path + fits filename (e.g., Cornish_data/HII/G010.8519-00.4407_I4.fits),
- reads the fits file using astropy.io.getdata,
- resizes the image,
- normalises the pixels values to be between 0 and 1,
- flattens the 2D images into 1D,
- returns the resized, normalised, flattened 1D array

You can use the [glob](https://docs.python.org/3/library/glob.html) function to return a list of filenames that can be looped over and read in by your function. You will need to stack each flattened image into a larger array that contains all 300 sources. In addition, you will need the corresponding 1D vector of labels, as well as the binarized version.

Finally, you will need to split your data into a training and testing/validation dataset (recall scikit learn's handy [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function).

## 1.4 Running the ANN

With your data prepared, you can now start building your ANN. Start off by building a Sequential ANN with the same architecture, optimizer, and hyperparameters as that used in [the ANN lectorial](https://github.com/MQ-ASTR3110/ASTR3110_Tutorial_Notebooks_2024/blob/main/Solution_Notebooks/ASTR3110_Tutorial_9_ANNs.ipynb), but modify the inputs so that they suit the data used here. 

## 1.5 Quantifying the performance

Once your model is trained, run a few predictions using the test data and compare with the known label. Produce a classification report using the test data.

<font color='blue'> **(1c)** Using the saved history from your model fit, plot on separate graphs the evolution of the Training and Testing/Validation loss and the evolution of the Training and Testing/Validation accuracy.</font> You can also use the plotting code from [Random Forest lectorial](https://github.com/MQ-ASTR3110/ASTR3110_Tutorial_Notebooks_2024/blob/main/Solution_Notebooks/ASTR3110_Tutorial_8_Random_Forest.ipynb) to produce a confusion matrix to help assess the classifier.

<font color='blue'> **(1d)** Based on the outputs of the classification report and the plots, assess the performance of the classifier. Please include the classification report in your document.</font>

## 1.6 Tweak the hyperparameters to improve performance

Try to improve your classifier by changing the following:

- The learning rate and number of epochs (smaller learning rate generally requires more training epochs and vice versa).
- The number of hidden layers (only try 1-2 fewer/more).
- The number of neurons in the layers (again, only try 1-2 different values).

<font color='blue'> **(1e)** For each tweak, explain and justify what was changed, run the classification reports, generate a confusion matrix, produce plots of the history of the Loss and Accuracy, and give a brief assessment of the performance.</font> 

<font color='blue'> **(1f)** Finally, give a summary report for the best classifier achieved, and outline which of the changes was the most effective.</font>

# <font color='red'> Aim to get up to here by the end of Lab session 1 </font>

# Part 2: Convolutional Neural Networks

In this part, you will build a classifier for the same dataset, but now using all three Spitzer bands (I2 = 4.5 $\mu$m, I3 = 5.8 $\mu$m, I4 = 8 $\mu$m) as input to a CNN. 


## Preparing the data

As input, CNNs take a 4D array of images with shape (N_source, N_pix, N_pix, N_channel), where  N_source is the number of sources in your batch of data, N_pix is the number of pixels (can be different for width/height), and N_channel is the number of different colour images available per source (this could be RGB channels for standard images, but here it is the 3 Spitzer bands). As for the ANNs, we also require labels that have been binarized. You need to write a function similar to the one used to manipulate the data for the ANN, but modified to produce the desired input for the CNN:
- For each source, read and then resize the I2, I3, and I4 images. Stack to form an array with shape (N_pix, N_pix, N_channel).
- Normalise each image in the stack, but here we'd like to maintain the colour differences. So, determine the maximum  across all 3 images and use that as your nomalisation factor for all three images.

In a similar fashion to Part 1, you will need to loop over each source, read in the images, and save to a 4D array with shape (N_source, N_pix, N_pix, N_Channel).  Again, you will require a vector containing the labels for each source, as well as the binarized version. 

Once you have the array containing the images and the labels, split your data into a training and test/validation set.

## Running the CNN
With your data prepared, you can now start building your CNN. Start off by bulding a Sequential CNN with the same architecture, optimizer, and hyperparameters as that used in [CNN Lectorial](https://github.com/MQ-ASTR3110/ASTR3110_Tutorial_Notebooks_2024/blob/main/Solution_Notebooks/ASTR3110_Tutorial_10_CNNs.ipynb), but **be aware that you will need to modify the inputs so that they suit the data used here.**

## Evaluate the performance of the CNN

Once your model is trained, run a few predictions using the test data and compare with the known label. Then, produce a classification report using the test data. 

<font color='blue'> **(2a)** Using the saved history from your model fit, plot on separate graphs the evolution of the Training and Testing/Validation loss and the evolution of the Training and Testing/Validation accuracy.</font> You can also use the plotting code from [the random forest lectorial](https://github.com/MQ-ASTR3110/ASTR3110_Tutorial_Notebooks_2024/blob/main/Solution_Notebooks/ASTR3110_Tutorial_8_Random_Forest.ipynb) to produce a confusion matrix to help assess the classifier. 

<font color='blue'> **(2b)** Based on the outputs of the classification report and the plots, assess the performance of the classifier. Please include the classification report in your document. </font>


## Tweak the CNN to improve performance

As before, try to improve your classifier by changing the following:

- The learning rate and number of epochs (smaller learning rate generally requires more training epochs and vice versa).
- The number of hidden layers (only try 1-2 extra).
- The number of neurons in the layers (again, only try 1-2 different values).
- You may also try changing the input resolution of the images.

<font color='blue'> **(2c)** For each tweak, explain and justify what was changed, run the classification reports, generate a confusion matrix, produce plots of the history of the Loss and Accuracy, and give a brief assessment of the performance. </font>

<font color='blue'> **(2d)** Finally, give a summary report for the best classifier achieved, and outline which of the changes was most effective. How did your best CNN classifier compare with your best ANN classifier?</font>

## Stretch Goal: Data Augmentation

It is possible to increase the size of our training set by manipulating the current data. For example, we can increase the size of our training set by rotating each image by 90, 180, and 270 degrees. Because our images of HII regions, PNE and RGs can have very similar shapes, but many different orientations on the sky, adding rotated images can help the CNN better learn our data. Aside from rotation, there are many other ways to augment the data, e.g., mirror-imaging, scaling the size, changing the perspective, and more.

Here, you can modify your training dataset using the ndimage.rotate function. In principle, you can add as many random orientations as you like, but I suggest that you start by rotating the three images for each source by 90, 180, and 270 degrees. You will need to also produce a new set of labels for your expanded dataset. 

<font color='blue'> **(2e)** Rerun your best CNN classifier on this new training set. Can you see an improvement in performance? (only required for PHYS7912 students)</font>
